<a href="https://colab.research.google.com/github/HIsu1231/NLP/blob/main/review_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4MB 29.5MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 460kB 39.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt
okt = Okt()

#morphs: 형태소 추출
print(okt.morphs("희수는 Hi라고 말했다."))

['희수', '는', 'Hi', '라고', '말', '했다', '.']


In [ ]:
#pos: 품사 태깅
print(okt.pos("희수는 Hi라고 말했다."))

[('희수', 'Noun'), ('는', 'Josa'), ('Hi', 'Alpha'), ('라고', 'Josa'), ('말', 'Noun'), ('했다', 'Verb'), ('.', 'Punctuation')]


In [ ]:
#nouns:명사 추출출
print(okt.nouns(""))

[]


In [ ]:
import pandas as pd

true = pd.read_csv('/content/drive/MyDrive/종프/total_true_review.csv', dtype='string')
fake = pd.read_csv('/content/drive/MyDrive/종프/total_fake_review.csv', dtype='string')

In [ ]:
T = pd.DataFrame(true["블로그내용"])
T["label"] = 1

F = pd.DataFrame(fake["블로그내용"])
F["label"] = 0

In [ ]:
cv = CountVectorizer(tokenizer=get_noun)
tdm = cv.fit_transform(F["블로그내용"])

cv.vocabulary_

NameError: ignored

In [ ]:
cv = CountVectorizer(tokenizer=get_noun)
tdm = cv.fit_transform(T["블로그내용"])

cv.vocabulary_

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


{'오늘': 3289,
 '포스팅': 5064,
 '크림': 4718,
 '내': 702,
 '통째': 4858,
 '거': 139,
 '진짜': 4281,
 '난': 671,
 '거나': 142,
 '일': 3718,
 '때': 1156,
 '가기': 4,
 '이': 3605,
 '제품': 4057,
 '알': 2978,
 '후': 5484,
 '마몽드': 1435,
 '라마': 1202,
 '이드': 3617,
 '인텐스': 3716,
 '정가': 4002,
 '흐트러진': 5518,
 '피부': 5171,
 '장벽': 3861,
 '강화': 108,
 '힘': 5542,
 '무궁화': 1651,
 '보습': 1987,
 '나': 631,
 '기획': 562,
 '세트': 2471,
 '인터파크': 3715,
 '아마': 2882,
 '할인': 5273,
 '정도': 4005,
 '번': 1915,
 '구매': 386,
 '아주': 2931,
 '형': 5378,
 '약간': 3046,
 '로션': 1312,
 '설명': 2424,
 '기준': 554,
 '화장': 5441,
 '전': 3940,
 '평소': 5049,
 '안': 2947,
 '토': 4824,
 '에센스': 3167,
 '발리': 1843,
 '톡톡': 4844,
 '수분': 2575,
 '흡수': 5525,
 '유지': 3534,
 '향': 5332,
 '꽃': 601,
 '부담': 2040,
 '닥터': 858,
 '아침': 2935,
 '방법': 1865,
 '정보': 4015,
 '올리브영': 3345,
 '쇼핑': 2556,
 '에이': 3177,
 '트루': 4887,
 '안심': 2965,
 '클렌징': 4730,
 '오일': 3311,
 '필라': 5198,
 '그린': 466,
 '배리': 1883,
 '밤': 1855,
 '테라': 4802,
 '브레스': 2124,
 '가글': 3,
 '마': 1418,
 '일드': 3726,
 '민트': 1761,
 '친구': 456

In [ ]:
data = T.append(F)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.33, random_state=42)

In [ ]:
X_train = train["블로그내용"]
y_train = train["label"]

X_test = test["블로그내용"]
y_test = test["label"]

In [ ]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer

def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf_svm = Pipeline([('vect', CountVectorizer(tokenizer=okt.morphs)),
                         ('tfidf', TfidfTransformer(use_idf=True)), #TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42))])

text_clf_svm = text_clf_svm.fit(X_train, y_train)

In [ ]:
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.9702276707530648

In [ ]:
text_clf_svm.predict(["요즘 날이 참 좋아졌어요 여러분~ 본격적으로 봄이 다가온 것 같아서 하루 빨리 일상생활로 복귀 되었으면 하는 작은 바람을 갖게 되는 요즘. 날은 따뜻해졌지만 일교차가 심해진만큼 민감하고 달아오른 피부 관리가 최근 저의 가장 큰 관심사가 아닐까 싶습니다 ㅎㅎ 그래서 오늘은 자극이 적고 피부 진정에 도움이 되는 토너를 한가지 소개하려고 하는데요! 최근에 정말 잘 쓰고있는 제품이라 라운드랩 1025 독도패드를 꼼꼼히 살펴 보도록 하겠습니다 :) 이름부터 정말 예쁜 독도패드! 저는 독도라는 단어를 보면 뭔가 뭉클한 그런 기분이 드는데요ㅎㅎ 이 안에 울릉도 해양심층수를 담아냈고 한 장으로 진정은 물론 각질 관리와 수분까지 채울 수 있는 제품이랍니다. 토너를 화장솜에 덜어서 사용하는 것보다 훨씬 쉽고 간편하게 쓸 수 있는 패드라서 피부를 관리할 때 훨씬 쉽게 할 수 있어요~ 인증받은 100% 순면이기 때문에 평소에 민감하고 자극에 예민한 피부도 안심하고 사용할 수 있구요~ PHA 성분을 더했기 때문에 쓸 때 자극이 덜합니다 ㅎㅅㅎ 제가 정말 최근에 피부가 많이 예민해져 조금만 자극적이면 금방 달아오르는데 요건 약산성이라 진짜 사용감이 편안하고 진정감도 좋아서 부담없이 쓰게 되더라구요! 피부에 잘 맞으니까 자꾸 손이 갑니다ㅎㅎ 70매라서 넉넉한 장수도 마음에 드는데 아침, 저녁으로 매일 사용하니까 생각보다 금방 쓰게 되더라구요 ㅠㅠ 아무래도 앞으로 쭉 쓸거니까 더 많이 사놔야겠단 생각을 하게 되는데요ㅋㅋ 이렇게 뚜겅을 열고 나서는 수분이 날아가지 않게 잘 닫아주시는게 중요합니다. 딱 보면 순면인데 보습력이 느껴지죠? 수분감을 가득 머금은 패드라서 맑고 투명한 느낌을 주는데요~ 이 에센스를 가득 머금은 패드가 피부를 매끈하고 촉촉하게 가꿔준답니다. 그리고 라운드랩 1025 독도패드 케이스에는 이렇게 패드에 대한 설명이 담겨 있는데요~ 양 면의 패드가 다르기 때문이랍니다! 이중 순면 패드는 각각의 활용도가 다른데, 탄탄한 거즈면은 각질, 노폐물 관리와 피부결을 정돈하는 용도로 사용하면 좋고 부드러운 순면은 피부 보습과 진정에 좋아요. 양 쪽 모두 사용하면서 케어하면 진짜 달라진 피부가 느껴진답니다 ㅎㅎ 저는 아침, 저녁에 다른 방법으로 사용하는데용 먼저 아침에는 진정팩 하듯이 얼굴 위에 올려놓고 밤사이 자극받은 피부를 편안하게 케어합니다. 이렇게 양 볼에 패드를 올린 다음에 옷 입고, 머리 말리면 딱 좋음! 그 사이에 피부 보습을 채워줍니다. 특히 평소에 베이스가 잘 뜨는 부분이 있다면 그 위에 올려 놓으셔도 좋은데요~ 저도 눈 밑쪽에 있는 볼 부분 화장이 잘 뜨는 편이라 이렇게 양 볼 위에 올리면 그 다음에 메이크업이 진짜 잘 먹더라구요 ㅎㅅㅎ! 저녁에는 세안을 마친 다음 스킨케어 첫단계에 사용하는데요~ 거즈면으로 피부결을 따라 가볍게 쓸어 닦아낸 다음에, 순면으로 한번 더 결정리를 하면 끝! 바깥 활동을 하면서 얼굴에 쌓인 각질이나 메이크업 잔여물을 깨끗히 걷어냅니다. 이렇게 하면 깔끔하게 피부결도 정리하며 매끈한 피부가 완성 됩니다 ㅎㅅㅎ 딱 보면 번들거리거나 답답한 느낌이 없죠? 피부가 아주 편하기 때문에 여러분들도 사용할 때 부담스럽지 않을거에요~ 이렇게 라운드랩 독도패드로 결정리 한 다음 에센스나 크림으로 마무리하면 다음날 피부가 굉장히 매끈해요! 전, 후를 비교하니 에센스로 채워진 촉촉한 피부가 보이실겁니다 ㅎㅎ 이 위에 다른 제품을 바르면 흡수력 굉장히 좋게 마무리되니까 스킨케어 첫단계는 라운드랩 독도패드가 정말 좋은 것 같아요 ㅎㅅㅎ 흡수가 빠르고 시트가 촉촉한 간편진정 독도패드, 라은드랩 1025 독도패드! 민감한 피부도 사용할 수 잇는 저자극 제품을 찾는다면 꼭 한번 써보시길 바라며! 건조하고 일교차 심한 날씨 때문에 피부 고민이 있다면 얼른 날려버리시길 바랍니당 그럼 안농 :) "])

array([0])

pandas.core.series.Series